# 0. Import Library

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

import scipy.stats

ModuleNotFoundError: No module named 'scipy'

![](https://onlinelibrary.wiley.com/cms/asset/ea1d3bd8-afd7-4914-b645-74d424b6690d/advs3654-fig-0002-m.jpg)

# 1. Input Data

In [3]:
df = pd.read_csv('data/fruit.csv')

# 2. Data Preprocessing

## 2.1 Exploratory Data Analysis (EDA)

In [4]:
df.head()

,size,fruit
0,27,lemon
1,35,orange
2,39,orange
3,21,lemon
4,22,lemon


In [5]:
df.shape

(10, 2)

In [6]:
df.describe()

,size
count,10.000000
mean,30.500000
std,8.540752
min,21.000000
25%,22.750000
50%,29.500000
75%,37.250000
max,45.000000


In [ ]:
df.dtypes

## 2.2 Data Cleaning

### - Missing Value

In [ ]:
df.isnull().sum()

### - Inconsistent Data

In [ ]:
df.dtypes

In [ ]:
df.groupby('fruit')['fruit'].count()

### - Outliers

In [ ]:
df.describe()

## 2.3 Data Transformation

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def labelEncode(data, columns):
    columns_ = []
    for i in columns:
        lb = LabelEncoder()
        lb_ = lb.fit_transform(data[i].astype(str))
        data[i+'_Encoded']=lb_
        columns_.append(i+'_Encoded')
    return columns_,lb

In [ ]:
columns = ['fruit']
enc_columns, lb=labelEncode(df, columns)

In [ ]:
df.sample(5)

In [ ]:
for feat in enc_columns:
    print(feat[:-8])
    print(' ',df[feat[:-8]].unique())
    print(' ',df[feat].unique())
    print('  -------')

## [Optional] Export เป็นไฟล์เมื่อทำ Data Cleaning เสร็จแล้ว

In [ ]:
#data.to_csv('รหัส.csv',index=False)
#data.to_excel('รหัส.xlsx',index=False)

## 2.4 ตรวจสอบค่าสหสัมพันธ์ของตัวแปร x, y

In [ ]:
df.corr()

In [ ]:
plt.rcParams['figure.figsize'] = 10,7 
sns.heatmap(df.corr(), annot=True);

## 2.5 การกำหนด Feature / Target

In [ ]:
DataMatrix = df.values
DataMatrix.shape

In [ ]:
feature_cols = ['size']
X = df[feature_cols].values
Y = df.fruit_Encoded.values

## 2.6 Data Preparation (แบ่งข้อมูลสำหรับ Training / Testing)

In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.4, random_state=0)

# 3. Modelling: Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

## 3.1 Training Data

In [ ]:
model = DecisionTreeClassifier(criterion='entropy')

In [ ]:
model.fit(X_Train, Y_Train)

In [ ]:
model.score(X, Y)

## 3.2 Predict Data

In [ ]:
y_predict = model.predict(X_Test)

In [ ]:
y_predict_class = lb.inverse_transform(y_predict)

In [ ]:
for res, label in zip(y_predict, y_predict_class):
    print('Result: {} - {}'.format(res, label))

## 3.3 การแสดงกราฟ Decision Tree

In [ ]:
import pydotplus
from sklearn.tree import export_graphviz
from IPython.display import Image

In [ ]:
data_feature_names = ['size']

In [ ]:
dot_data = export_graphviz(model, out_file=None, 
                           feature_names=data_feature_names,  
                           class_names=y_predict_class,  
                           filled=True,
                           rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())   
graph.write_png("dtree_fruit.png")

In [ ]:
from IPython.display import display
from PIL import Image

path="dtree_fruit.png"
display(Image.open(path))

# 4. Model Evaluation

![](https://scontent.fbkk29-6.fna.fbcdn.net/v/t1.6435-9/99275955_3331806336852736_3844604936671199232_n.png?_nc_cat=109&ccb=1-7&_nc_sid=7f8c78&_nc_eui2=AeEPpdFqjY0zIBpWW6O52JXQCwjvKa4EPMcLCO8prgQ8x2YhylNrAmxFW2vPQB1sm2JXvyP_u9jYqj85NHKBdyh5&_nc_ohc=-XjmPNGvhvUAX_I9Z5f&_nc_ht=scontent.fbkk29-6.fna&oh=00_AfBY6aE1We6FnZidwobWCxZjP-uEUowhlA39STrUEJym2g&oe=6502682A)

### 4.1 Split Test

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(Y_Test, y_predict)
cnf_matrix

In [ ]:
plt.rcParams['figure.figsize'] = 10,7 
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cnf_matrix, display_labels = ['lemon', 'orange'])
cm_display.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['lemon', 'orange']
print(classification_report(Y_Test, y_predict, target_names=target_names))

### 4.2 Cross Validation Test

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cvs = cross_val_score(model, X, Y, cv=2)

In [ ]:
print('Cross Validation Score {}'.format(cvs))

In [ ]:
print('Mean: {:4f}'.format(cvs.mean()))

# [Optional] Save Machine Learning Models

In [ ]:
#import pickle
#filename = 'pickle_รหัสนศ.sav'
#pickle.dump(model, open(filename, 'wb'))